In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()
b = chat.predict("How many planets are there?")

b

/Users/cone/Documents/workspace/gpt/env/lib/python3.13/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/cone/Documents/workspace/gpt/env/lib/python3.13/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


('There are eight officially recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. There is ongoing debate and research on whether there are more planets beyond our solar system.',
 'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')